In [2]:
!pip install open_clip_torch

In [3]:
import pandas as pd
from PIL import Image
import torch
from open_clip import create_model_from_pretrained, get_tokenizer


In [4]:
model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# Load CSV file containing image-text pairs
df = pd.read_csv('/content/drive/MyDrive/Biomedclip/amboss_image_data_all_results.csv')


In [9]:
import os
image_embeddings = []
for index, row in df.iterrows():
    image_path = os.path.join("/content/drive/MyDrive/Biomedclip",row['subfolder'], f"{row['id']}.jpg")
    # Load and preprocess the image
    image = Image.open(image_path)
    image = preprocess(image)
    image_tensor = torch.Tensor(np.array(image))

# Add batch dimension to the image
    image_tensor = image_tensor.unsqueeze(0)
    # Compute embeddings for the image
    with torch.no_grad():
        image_embedding = model.encode_image(image_tensor)
    #text_tokens = tokenizer(row['hint'])
    #text_embedding = model.encode_text(text_tokens)
    image_embeddings.append((image_embedding, row['id']))

In [14]:
import pickle

# Store image embeddings to a file
with open('/content/drive/MyDrive/Biomedclip/image_embeddings.pkl', 'wb') as f:
    pickle.dump(image_embeddings, f)


In [24]:

# Retrieve image embeddings from file
with open('/content/drive/MyDrive/Biomedclip/image_embeddings.pkl', 'rb') as f:
    image_embeddings = pickle.load(f)

# Now image_embeddings contains the list of image embeddings along with their corresponding IDs


In [25]:
query_image = Image.open('/content/drive/MyDrive/Biomedclip/query_image_2.jpeg')
query_image = preprocess(query_image)

In [26]:
import torch
import numpy as np
# Convert PIL image to PyTorch tensor
query_image_tensor = torch.Tensor(np.array(query_image))

# Add batch dimension to the image
query_image_tensor = query_image_tensor.unsqueeze(0)

# Now you can pass the reshaped image tensor to the encode_image method

with torch.no_grad():
    query_image_embedding = model.encode_image(query_image_tensor)

In [27]:
similarities = []
for image_embedding,id in image_embeddings:
    similarity_score = torch.cosine_similarity(query_image_embedding, image_embedding)
    similarities.append((id, similarity_score))

# Sort pairs based on similarity and retrieve top k pairs
k = 3
top_k_pairs = sorted(similarities, key=lambda x: x[1], reverse=True)[:k]

# Print top k pairs
for pair in top_k_pairs:
    print(f"Image: {pair[0]}, Similarity Score: {pair[1]}")

Image: 716, Similarity Score: tensor([0.8072])
Image: 3, Similarity Score: tensor([0.8063])
Image: 891, Similarity Score: tensor([0.7958])


In [28]:
for pair in top_k_pairs:
  my_id_value = pair[0]
  hint_value = df.loc[df['id'] == my_id_value, 'hint'].values[0]
  print(hint_value)

The hand is covered with light brown papules and rough, hyperkeratotic plaques. This condition is the result of years of exposure to sunlight without protection.
A rash on the hands, feet, and mouth is typical of this pediatric febrile exanthem.
This patient's history of extensive sun exposure and hyperpigmented macules and patches are most consistent with solar lentigo.
